<a href="https://colab.research.google.com/github/ZonaQuant/Notebooks/blob/main/Backtest_medias.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install yahoo_fin yfinance pandas numpy

     |████████████████████████████████| 63 kB 1.1 MB/s 
     |████████████████████████████████| 6.4 MB 6.8 MB/s 
  Attempting uninstall: lxml
    Found existing installation: lxml 4.2.6
    Uninstalling lxml-4.2.6:
      Successfully uninstalled lxml-4.2.6
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests~=2.23.0, but you have requests 2.27.1 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [ ]:
import yahoo_fin.stock_info as si
import pandas as pd
import numpy as np
import yfinance as yf

tickers = si.tickers_sp500()
resumen=pd.DataFrame(columns=['Ticker','Mean 25',' %', 'Mean 50',' %','Mean 100', ' %','Mean 200',' %' ])

capital=float(100000)
cf=0.5
cv=0.0003  #0.3 por mil
ct=float(0)# Total comisiones por ticker y estrategia  (cf+cv)
tit=int()
resultado=float()

for i in range(len(tickers)):
    df =yf.download(tickers[i],'2021-01-01', progress=False)['Close']
    resumen.loc[i,'Ticker']= tickers[i]
    """ Calculamos distintas media """
    mean25 = df.rolling(25).mean()
    mean50 = df.rolling(50).mean()
    mean100 = df.rolling(100).mean()
    mean200 = df.rolling(200).mean()
    x=1 # controlamos columna importe y % a modo de contador
    p=0
    while p<4:
        cash=capital
        if p==0:
            data = pd.DataFrame(np.where((mean25 < 0.98*df),1,0), index=df.index)
        elif p==1:
            data = pd.DataFrame(np.where((mean50 < 0.97*df),1,0), index=df.index)
        elif p==2:
            data = pd.DataFrame(np.where((mean100 < 0.96* df),1,0), index=df.index)
        else:
            data = pd.DataFrame(np.where((mean200 < 0.95*df),1,0), index=df.index)

        """ Usamos un solo objeto para las 4 estrategias, se controla mediante el valor de p"""
        signal = data.diff()
        signal['Titulos']=0
        signal['Coste']=0
        signal['Venta']=0
        signal['Comision']=0

        for z in range(len(df)):
            """ Dada la condición de compra calculamos qué cantidad podemos comprar"""
            if signal.iloc[z,0]>0: # señal de compra
                signal.iloc[z,1]= cash//df[z] # numero de acciones
                tit+=signal.iloc[z,1] # acumulamos títulospq no sabemos cuando se venderán y vamos a borrar registros a cero
                signal.iloc[z,2]=round(signal.iloc[z,1]*df[z],2) # coste de la compra
                signal.iloc[z,4 ]= round(cf+signal.iloc[z,2]*cv,2)
                ct=ct+cf+signal.iloc[z,2]*cv
                cash+=-signal.iloc[z,2]-signal.iloc[z,4] # mantenemos acumulado cash neto
            
            if signal.iloc[z,0]<0 : # señal de venta
                signal.iloc[z,3]= round(tit *df[z],2) # importe venta
                signal.iloc[z,4]= round(cf+signal.iloc[z,3]*cv,2)
                ct=ct+cf+signal.iloc[z,2]*cv
                signal.iloc[z,1]=-tit # cuenta títulos
                tit=0
                cash+=signal.iloc[z,3]-signal.iloc[z,4]

        """ Borramos registros sin señal de compra o venta de ahí haber guardado titulos y cash fuera del DataFrame"""    
        signal=signal.fillna(0)
        signal=signal.loc[(signal!=0).any(axis=1)]
        """ Pantallazo de la estrategia, se podrá anular."""
        print('Ticker:',tickers[i],'Estrategia:',p)
        print(signal)
        ct=0
        resultado=round(float(tit*df[-1:] + cash),2) 
        porcentaje=round((resultado/capital-1)*100,2)
        print('Resultado: ',resultado, ' ', porcentaje,' %' )
        """Guardamos en resumen para lista objetivo , manipulamos x e y para lograr encolumnarlos correctamente"""
        resumen.iloc[i,(x)]=resultado
        resumen.iloc[i,(x+1)]=porcentaje
        tit=0
        x+=2
        p+=1
    print(resumen)# Pantallazo del avance título a título, se podrá eliminar.

Se han truncado las últimas 5000 líneas del flujo de salida.
2021-04-27 -1.0   -142.0       0.00  100073.08     30.52
2021-06-14  1.0    162.0  100065.78       0.00     30.52
2021-06-15 -1.0   -162.0       0.00   97096.32     29.63
2021-06-17  1.0    157.0   96806.20       0.00     29.54
2021-07-07 -1.0   -157.0       0.00  101210.05     30.86
2021-07-08  1.0    155.0  101185.55       0.00     30.86
2021-07-14 -1.0   -155.0       0.00  101273.90     30.88
2021-07-29  1.0    149.0  100925.15       0.00     30.78
2021-08-16 -1.0   -149.0       0.00  102239.33     31.17
2021-08-23  1.0    145.0  102413.50       0.00     31.22
2021-08-26 -1.0   -145.0       0.00  101668.20     31.00
2021-08-27  1.0    143.0  101804.56       0.00     31.04
2021-09-20 -1.0   -143.0       0.00  104414.31     31.82
2021-09-22  1.0    139.0  104519.66       0.00     31.86
2021-11-15 -1.0   -139.0       0.00  140861.21     42.76
2021-11-16  1.0    133.0  140279.09       0.00     42.58
2021-11-26 -1.0   -133.0   